# Getting Started

In [1]:
import os
import numpy as np
import pandas as pd
import datetime
import re

import numpy as np
import pandas as pd
import string
import re
import matplotlib.pyplot as plt
import math
from matplotlib import rc
import random

from collections import Counter, defaultdict
import matplotlib.ticker as ticker
from math import sqrt

import plotly.express as px 
import matplotlib.pyplot as plt
import warnings

In [2]:
datetime.datetime.now()

datetime.datetime(2022, 6, 27, 17, 23, 34, 577340)

In [3]:
cols = ["song", "songID", "artist", "album", "genre", "play_count", "click_count"]
# df_test = pd.read_csv('C:/Users/awrg/Documents/Delft/Thesis/Projects/Datasets/Engineered/test_dataset_small_1.csv')
df_test = pd.read_csv('C:/Users/awrg/Documents/Delft/Thesis/Projects/Datasets/Engineered/test_dataset_small_1.csv')
df_log = pd.read_csv('C:/Users/awrg/Documents/Delft/Thesis/Projects/Datasets/Engineered/test_clicklog_small_v1.csv', index_col=0)
df_log2 = pd.read_csv('C:/Users/awrg/Documents/Delft/Thesis/Projects/Datasets/Engineered/test_clicklog_full_v1.csv', index_col=0)    

In [4]:
df_test

,song,songID,artist,album,genre,play_count,click_count
0,Soul Food,1,Mat Zo,Self Assemble,Electronic,82294,555
1,The Enemy,2,Mat Zo,Self Assemble,Electronic,42875,777
2,Everytime We Touch,3,Cascada,Everytime We Touch,Electronic,1756387293,9872342342
3,Something in the Way,4,Nirvana,Nevermind,Grunge,171857763,223746531
4,heat,5,53 Thieves,after hours,Electronic,88342,101527
5,Another Life,6,Jadu Heart,Hyper Romance,Electronic,17009,28032
6,Numb,7,Elderbrook,Why Do We Shake In The Cold?,Electronic,441070,565088
7,Control,8,Emmit Fenn,The Last Dance,Electronic,267650,310823
8,Fire for You,9,Cannons,Shadows,Electronic,32199,37825
9,Dope,10,Demeter,Dope,Electronic,1780336,2138853


In [5]:
df_log

,userID,user_query,click_position,title,magnet
0,5,soul food,2,Soul Food,link0
1,5,Mat Zo,11,The Enemy,link1
2,7,Swedish dance song,3,Everytime We Touch,link2
3,1,Nirvana Batman song,1,Something in the Way,link3
4,3,heat,5,heat,link4
5,4,Another Life Jadu Heart,1,Another Life,link5
6,9,Numb by Elderbrook,1,Numb,link6
7,8,Control,17,Control,link7
8,8,Cannons Fire for You,1,Fire for You,link8
9,8,dope,11,Dope,link9


In [6]:
df_log2

,title,artist,magnet,click_count
0,Marvu – Underground Jazz [2010] [EP],Marvu,magnet:?xt=urn:btih:77efa41fc2eb77996b2fd2d2e7...,771501
1,Is World – Turning [2010] [EP],Is World,magnet:?xt=urn:btih:593643871726261298e4806a64...,991597
2,...And Stars Collide – ...And Stars Collide [2...,...And Stars Collide,magnet:?xt=urn:btih:077293393f826c6bc1ccf8ff53...,833324
3,Sadsic – [sun] [2009] [EP],Sadsic,magnet:?xt=urn:btih:ad04b4cab2d25322b55f941ad6...,970405
4,Boreals – Rome [2011] [EP],Boreals,magnet:?xt=urn:btih:2f515edefd669e6e1801a00160...,299206
...,...,...,...,...
248,The Jim Tablowski Experience – SHOULD WE MAKE ...,The Jim Tablowski Experience,magnet:?xt=urn:btih:a2acdfda4a9da275f17bedbde2...,885452
249,Commonplaces – A private sea [2011] [Album],Commonplaces,magnet:?xt=urn:btih:8ed2e212fa90d01b91c9f27ea7...,29167
250,Maharajah with Charl and H-sie – Good Hearted ...,Maharajah,magnet:?xt=urn:btih:cdceb58b04cd7bc285a5caeb6d...,716629
251,Big Joe Daddy – 11:11 [2014] [Album],Big Joe Daddy,magnet:?xt=urn:btih:345699cc10bdb566307f5d48f8...,206696


In [7]:
# df_log = pd.DataFrame(columns=['userID', 'user_query', 'click_position', 'title', 'magnet'])
# df_log.userID = np.zeros(10)
# df_log.user_query = ['soul food', 'Mat Zo', 'Swedish dance song', 'Nirvana Batman song', 'heat', 'Another Life Jadu Heart', 'Numb by Elderbrook', 'Control', 'Cannons Fire for You', 'dope']
# df_log.title = df_test.song.copy()
# df_log.click_position = [2, 11, 3, 1, 5, 1, 1, 17, 1, 11]
# df_log.userID = [5, 5, 7, 1, 3, 4, 9, 8, 8, 8]
# df_log.magnet = ['link0', 'link1', 'link2', 'link3', 'link4', 'link5', 'link6', 'link7', 'link8', 'link9']

# df_log.to_csv('C:/Users/awrgo/Documents/Delft/Thesis/Projects/Datasets/Engineered/test_clicklog_small_v1.csv')

# url = 'https://raw.githubusercontent.com/brian2509/pandacd-scrape/master/export/220408/assets/output.csv'
# df_log2 = pd.read_csv(url)

# df_log2['click_count'] = np.random.randint(0, 1000000, df_log2.shape[0])

# df_log2.to_csv('C:/Users/awrg/Documents/Delft/Thesis/Projects/Datasets/Engineered/test_clicklog_full_v1.csv')




# df_log2

---
# Global Objects and Classes

Nodes, Networks, etc

In [8]:
class Node:

    nodeID = None
    library = None
    clicklog = None
    
    def __init__(self, newID):
        self.nodeID = newID
        self.library = pd.DataFrame()
        self.clicklog = pd.DataFrame()
        
    
    def updateLibrary(self, newData):
        self.library = self.library.append(newData, ignore_index=True)
        
        
    def gossip_library(self, neighbors, to_gossip, debug):
        
        timestamp = datetime.datetime.now()
        for n in neighbors:
#             message = Message(this.nodeID, n.nodeID, to_gossip, timestamp)
            temp = n.library
            n.library = pd.concat([n.library, to_gossip]).drop_duplicates()
            n.library.reset_index(inplace=True, drop=True)
            if(debug): 
                print("Added", to_gossip)
                print(" to", temp)
                print("creating: ", n.library)
            temp = None
        

In [9]:
class Message:
    
    fromID = None
    toID = None
    contents = None
    timestamp = None
    
    
    def __init__(self, fromID, toID, contents, timestamp):
        self.fromID = fromID
        self.toID = toID
        self.contents = contents

In [143]:
class Network:
    
    nodes = None
    default_cols = None    
    library = None
    clicklog = None
    
    
    def __init__(self):
        self.default_cols = pd.Series()
        self.library = pd.DataFrame()
        self.nodes = pd.Series()
        self.clicklog = pd.DataFrame()
    

    def __init__(self, numNodes, default_cols, default_df, default_log):
        self.default_cols = default_cols
        self.library = default_df
        self.nodes = []
        self.clicklog = default_log
        
        for i in range(numNodes):
            
            newNode = Node(i)
            newNode.library = self.library
            newNode.clicklog = self.clicklog
            self.nodes.append(newNode)
            
            
    def __init__(self, numNodes, default_df, default_log):
        self.library = default_df
        self.nodes = []
        self.clicklog = default_log
        
        for i in range(numNodes):
            
            newNode = Node(i)
            newNode.library = self.library
            newNode.clicklog = self.clicklog
            self.nodes.append(newNode)

    
    
#     def gossip(self, num_nodes):
        


In [11]:
def buildNetwork(numNodes, default_df):
    
#     network = Network()
    network = []
    
    for i in range(numNodes):
        
        newNode = Node(i)
        newNode.library = default_df.copy()
        network.append(newNode)
        
    return network

In [12]:
def buildNetwork_v2(numNodes, default_df):
    
    network = []
    
    for i in range(numNodes):
        
        newNode = Node(i)
        newNode.library = default_df.sample(frac=np.random.rand(), axis=0, ignore_index=True).copy()
        network.append(newNode)
        
    return network

---
# Major Algorithms

- Search
    - (v1) Must be able to return all string matches
    - Ideally ranks based on popularity (clicklog)
    - No user profiling/machine learning (yet)

- Gossip
    - Pass messages occasionally (when/why?)
    - 


- Rank

### Search


In this iteration, I will assume that the first node that it finds the result in is also the most efficient to reach, therefore I will only return the location of the first result (instead of a list of all containing nodes).

Problem with this is: what if each node has a different dataset? We can't end the search at just the first hit. We need to go through everything nearby. **Therefore I think it's better to keep a list of all contained nodes, and limit the search in some other way to constrain it**.


In [13]:
# debug = False

# def search(term, to_search):
    
#     # We need to aggregate the results where we have a match
#     total_results = pd.DataFrame()
#     contained_nodes = []
#     first_found = -1
    
#     # Look through the list of nodes to search through
#     for node in to_search:
#         if(debug): print("node:", node.nodeID)
        
#         # For each column in the library of each node
#         for col in node.library.columns:
                
#             # Get the columns of where the term is found in the node library
#             node_results = node.library.iloc[np.where(node.library[col].astype(str).str.find(term) == 0)]
            
#             if(debug): 
#                 print("column:", col)
#                 print("node_results:", node_results)
            
#             # Concatenate the total results into a dataframe
#             total_results = pd.concat([node_results, total_results]).drop_duplicates()
            
#             # Keep track of the nodes that the result is found in
#             if(len(node_results) > 0):
#                 print("node", node.nodeID, "contained the search term:", term)
#                 total_results["found_in"] = node.nodeID
                
       
#     return total_results

### Search/rank v1:

- Search all nodes
- Return all literal string matches
    - including a list of every node it was found on
- Sort the matches based purely on click count

In [14]:
####
# This iteration collects all nodes that the result is found in
####


debug = False

def search(term, to_search):
    
    term = term.lower()
    
    # We need to aggregate the results where we have a match
    total_results = pd.DataFrame()
    contained_nodes = []
    
    
    # Look through the list of nodes to search through
    for node in to_search:
        if(debug): print("node:", node.nodeID)
        
        # For each column in the library of each node
        for col in node.library.columns:
                
            # Get the columns of where the term is found in the node library
            node_results = node.library.iloc[np.where(node.library[col].astype(str).str.contains(term, na=False, case=False))]
            
            if(debug): 
                print("column:", col)
                print("node_results:", node_results)
            
            # Concatenate the total results into a dataframe
            total_results = pd.concat([node_results, total_results]).drop_duplicates()
            
            # Keep track of the nodes that the result is found in
            if(len(node_results) > 0):
                if(debug): print("node", node.nodeID, "contained the search term:", term)
                contained_nodes.append(node.nodeID)
       
    
    # add the list of contained nodes to the returned dataset
#     for i in range(len(total_results)):
#         total_results['contained_nodes'] = []
#         total_results.assign(contained_in = [contained_nodes])     

    if(debug): print("contained nodes:", contained_nodes)
        
    total_results = total_results.reset_index(drop=True, inplace=False)

    return ranked_results(total_results), contained_nodes


def ranked_results(results):
    
    return results.sort_values(by='click_count', ascending=False)

### Search and Rank

This goes beyond just ranking based on click. How?

- ?
  

In [15]:
def searchAndRank(term, toSearch):
    
    
    
    return rank_v1(search_results), contained_nodes

### Gossip

In [79]:
numNodes = 25
network = buildNetwork_v2(numNodes, df_test)

TypeError: buildNetwork_v2() takes 2 positional arguments but 3 were given

In [17]:
network[0].library

,song,songID,artist,album,genre,play_count,click_count
0,Fire for You,9,Cannons,Shadows,Electronic,32199,37825
1,heat,5,53 Thieves,after hours,Electronic,88342,101527


In [18]:
max_n = 3

for node in network:
    
    neighbors = random.sample(network, int(np.random.rand() * max_n))
    node.gossip_library(neighbors, node.library, False)

In [19]:
network[7].library

,song,songID,artist,album,genre,play_count,click_count
0,Everytime We Touch,3,Cascada,Everytime We Touch,Electronic,1756387293,9872342342
1,Fire for You,9,Cannons,Shadows,Electronic,32199,37825
2,Another Life,6,Jadu Heart,Hyper Romance,Electronic,17009,28032
3,Dope,10,Demeter,Dope,Electronic,1780336,2138853
4,Soul Food,1,Mat Zo,Self Assemble,Electronic,82294,555


---
# Testing Environment

#### Partial network knowledge

This example has N nodes with partial knowledge of the global dataset, and use "gossip" to learn more about the global state and expand their local dataset.

In [20]:
numNodes = 10

# build network where each node has the full dataset
# network = buildNetwork(numNodes, df_test)


# build network where each node has 25% of the full dataset
network = buildNetwork_v2(numNodes, df_test)

In [21]:
search("Electronic", network)

(                 song  songID      artist                         album  \
 2  Everytime We Touch       3     Cascada            Everytime We Touch   
 5                Dope      10     Demeter                          Dope   
 3                Numb       7  Elderbrook  Why Do We Shake In The Cold?   
 0             Control       8  Emmit Fenn                The Last Dance   
 1                heat       5  53 Thieves                   after hours   
 7        Fire for You       9     Cannons                       Shadows   
 6        Another Life       6  Jadu Heart                 Hyper Romance   
 4           The Enemy       2      Mat Zo                 Self Assemble   
 8           Soul Food       1      Mat Zo                 Self Assemble   
 
         genre  play_count  click_count  
 2  Electronic  1756387293   9872342342  
 5  Electronic     1780336      2138853  
 3  Electronic      441070       565088  
 0  Electronic      267650       310823  
 1  Electronic       88342  

In [22]:
node = network[0]

df_test.iloc[np.where(node.library.song.astype(str).str.contains("you", na=False, case=False))]

,song,songID,artist,album,genre,play_count,click_count


In [179]:
# Basic search seems to work
search("the", network.nodes)[0]

,song,songID,artist,album,genre,play_count,click_count
3,Something in the Way,4,Nirvana,Nevermind,Grunge,171857763,223746531
0,Numb,7,Elderbrook,Why Do We Shake In The Cold?,Electronic,441070,565088
1,Control,8,Emmit Fenn,The Last Dance,Electronic,267650,310823
4,Another Life,6,Jadu Heart,Hyper Romance,Electronic,17009,28032
2,The Enemy,2,Mat Zo,Self Assemble,Electronic,42875,777


In [184]:
qwe = search('the', network.nodes)[0].sample()
qwe

,song,songID,artist,album,genre,play_count,click_count
3,Something in the Way,4,Nirvana,Nevermind,Grunge,171857763,223746531


In [177]:
wer = pd.DataFrame()

In [181]:


wer = wer.append(qwe)

wer

C:\Users\awrg\AppData\Local\Temp\ipykernel_7888\1827203509.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  wer = wer.append(qwe)


,song,songID,artist,album,genre,play_count,click_count
0,Numb,7,Elderbrook,Why Do We Shake In The Cold?,Electronic,441070,565088
1,Control,8,Emmit Fenn,The Last Dance,Electronic,267650,310823


---
### Global dataset knowledge

This example has N nodes, all of which have global knowledge of the entire state of the network and all its contents, including the global dataset of music and clicklog(s).

#### Search and Click

Create a simulation of searching and clicking:

- Network is created with global dataset
- Random nodes start searching for terms
- Once search is performed, they click on (random entry)
- Global clicklog is updated
    
    
- **Next step: update the search based on clicklog, first example of L2R**

In [297]:
numNodes = 10

empty_log = pd.DataFrame(columns=df_log.columns)
network = Network(numNodes, df_test, empty_log)


In [298]:
network.clicklog.columns[0]


'userID'

In [299]:
corpus = df_test.song.drop_duplicates(inplace=False)
corpus = corpus.append(df_test.artist).drop_duplicates(inplace=False)
corpus = corpus.append(df_test.album).drop_duplicates(inplace=False)
corpus = corpus.append(df_test.genre).drop_duplicates(inplace=False).reset_index(inplace=False, drop=True)


# corpus

C:\Users\awrg\AppData\Local\Temp\ipykernel_7888\3203682018.py:2: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  corpus = corpus.append(df_test.artist).drop_duplicates(inplace=False)
C:\Users\awrg\AppData\Local\Temp\ipykernel_7888\3203682018.py:3: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  corpus = corpus.append(df_test.album).drop_duplicates(inplace=False)
C:\Users\awrg\AppData\Local\Temp\ipykernel_7888\3203682018.py:4: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  corpus = corpus.append(df_test.genre).drop_duplicates(inplace=False).reset_index(inplace=False, drop=True)


In [300]:
# number of random searches to be done

num_searches = 100


for i in range(num_searches):
    
    # randomly choose a node to be searching
    searching_node = np.random.randint(0,len(network.nodes))
    
    # randomly choose a term to search for from the corpus
    term = corpus.sample().values[0]
    
    # compile all results
    results = search(term, network.nodes)
    
    # randomly choose an item from the results. Search returns all results from all nodes. Pick the first, randomly sample
    clicked = results[0].sample()
    
#     print(clicked)
    
    to_append = {network.clicklog.columns[0]: network.nodes[searching_node].nodeID,
                network.clicklog.columns[1]: term,
                network.clicklog.columns[2]: np.random.randint(1,network.library.shape[0]),
                network.clicklog.columns[3]: clicked.song,
                network.clicklog.columns[4]: None}
    
    to_append = pd.DataFrame(to_append, index=[0])
    
    
    network.clicklog = pd.concat([network.clicklog, to_append], axis=0, ignore_index=False)


In [301]:
network.clicklog

,userID,user_query,click_position,title,magnet
0,5,Grunge,7,Something in the Way,None
0,5,Emmit Fenn,7,Control,None
0,6,Something in the Way,8,Something in the Way,None
0,2,Something in the Way,3,Something in the Way,None
0,5,Demeter,9,Dope,None
...,...,...,...,...,...
0,9,Fire for You,1,Fire for You,None
0,7,Dope,8,Dope,None
0,0,Hyper Romance,7,Another Life,None
0,6,Control,9,Control,None


In [282]:
corpus.sample().values[0]

'Dope'

In [283]:
np.random.randint(1,network.library.shape[0])

4

In [284]:
network.nodes

In [127]:
# Basic search seems to work
search("the", network.nodes)[0].sample()

,song,songID,artist,album,genre,play_count,click_count
4,Another Life,6,Jadu Heart,Hyper Romance,Electronic,17009,28032


---
# Methods and code to save for later

In [33]:
# Lambda expression to see if a term is in a column of a dataframe
df_test.apply(lambda row: row.astype(str).str.contains('Mat Zo').any(), axis=1)

0     True
1     True
2    False
3    False
4    False
5    False
6    False
7    False
8    False
9    False
dtype: bool

In [34]:
# Regex to ignore case in string search, can only compare two string objects though (afaik)
re.search("soul", df_test.iloc[0].song, re.IGNORECASE)

<re.Match object; span=(0, 4), match='Soul'>